In [2]:
import json
with open("DBLP-QuAD/DBLP-QuAD/test/questions.json", "r", encoding="utf-8") as f:
    dblp_data = json.load(f)

In [3]:
with open("DBLP-QuAD/DBLP-QuAD/test/answers.json", "r", encoding="utf-8") as f:
    dblp_answers = json.load(f)

In [60]:
dblp_answers['answers'][0]

{'id': 'Q0001',
 'answer': {'head': {'link': [], 'vars': ['answer']},
  'results': {'distinct': False,
   'ordered': True,
   'bindings': [{'answer': {'type': 'uri',
      'value': 'https://www.wikidata.org/entity/Q55238282'}}]}}}

In [3]:
from utils import question_template

question_template("SciQA-dataset/SciQA-dataset/test/questions.json", "SciQA-dataset/SciQA-dataset/test_question_templates.json")

In [1]:
import openai
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("UNI_API_KEY")

client = openai.OpenAI(
    api_key=api_key,
    base_url="https://llms-inference.innkube.fim.uni-passau.de" )


response = client.chat.completions.create(
    model="llama3.1",
    messages = [
        {
            "role": "user",
            "content": "I don't have information about the current CEO of Intel Corp. Last information: Pat Gelsinger from 15.02.21 to 01.12.24. Could you provide an up-to-date information?"
        }
    ]
)

print(response)

ChatCompletion(id='chatcmpl-295c5d23-2198-4a50-93c2-581706521a3e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='As of my last update in April 2023, the information regarding the CEO of Intel Corporation is as follows:\n\n- **Pat Gelsinger** served as the CEO from February 15, 2021. However, I do not have real-time updates or changes that might have occurred after my last knowledge cutoff.\n\nTo get the most current and accurate information, I recommend checking the official Intel Corporation website, recent news releases, or reputable financial news sources for any updates on leadership positions within the company.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1753352432, model='ollama/llama3.1:70b-instruct-q4_0', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=102, prompt_tokens=60, total_tokens=162, co

In [ ]:
from generate_prompt import get_similar_questions_bert
import json

with open('SciQA-dataset/SciQA-dataset/train/questions.json', 'r') as f:
    q_templates = json.load(f)["questions"]



c:\Users\Administrator\kg-qa\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[('List the title and ID of research papers that contain a benchmark over the SVHN dataset?',
  'SELECT DISTINCT ?paper ?paper_lbl\nWHERE {\n  ?dataset        a                orkgc:Dataset;\n                  rdfs:label       ?dataset_lbl.\n  FILTER (str(?dataset_lbl) = "SVHN")\n  ?benchmark      orkgp:HAS_DATASET       ?dataset.\n  ?cont          orkgp:HAS_BENCHMARK      ?benchmark.\n  ?paper         orkgp:P31                ?cont;\n                 rdfs:label               ?paper_lbl.\n}'),
 ('List the title and ID of research papers that contain a benchmark over the LAMBADA dataset?',
  'SELECT DISTINCT ?paper ?paper_lbl\nWHERE {\n  ?dataset        a                orkgc:Dataset;\n                  rdfs:label       ?dataset_lbl.\n  FILTER (str(?dataset_lbl) = "LAMBADA")\n  ?benchmark      orkgp:HAS_DATASET       ?dataset.\n  ?cont          orkgp:HAS_BENCHMARK      ?benchmark.\n  ?paper         orkgp:P31                ?cont;\n                 rdfs:label               ?paper_lbl.\n}

In [2]:
question = "What models are being evaluated on the UrbanSound8k dataset?"
get_similar_questions_bert(question, q_templates)

[('What models are being evaluated on the CIFAR-10 dataset?',
  'SELECT DISTINCT ?model ?model_lbl\nWHERE {\n  ?dataset       a                orkgc:Dataset;\n                  rdfs:label       ?dataset_lbl.\n  FILTER (str(?dataset_lbl) = "CIFAR-10")\n  ?benchmark      orkgp:HAS_DATASET       ?dataset;\n                  orkgp:HAS_EVALUATION    ?eval.\n  ?paper         orkgp:HAS_BENCHMARK      ?benchmark.\n  OPTIONAL {?paper      orkgp:HAS_MODEL          ?model.\n            ?model      rdfs:label               ?model_lbl.}\n}'),
 ('What models are being evaluated on the BC2GM dataset?',
  'SELECT DISTINCT ?model ?model_lbl\nWHERE {\n  ?dataset       a                orkgc:Dataset;\n                  rdfs:label       ?dataset_lbl.\n  FILTER (str(?dataset_lbl) = "BC2GM")\n  ?benchmark      orkgp:HAS_DATASET       ?dataset;\n                  orkgp:HAS_EVALUATION    ?eval.\n  ?paper         orkgp:HAS_BENCHMARK      ?benchmark.\n  OPTIONAL {?paper      orkgp:HAS_MODEL          ?model.\n  

In [8]:
len(dblp_answers['answers'][1]['answer']['results']['bindings'])

187

In [4]:
prefixes="""
PREFIX dblp: <https://dblp.org/rdf/schema#>
PREFIX cito: <http://purl.org/spar/cito/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>
"""

question = dblp_data["questions"][1]

query_body = question["query"]["sparql"]

full_query = prefixes + "\n" + query_body

In [ ]:
import json
import re
from sklearn.metrics import accuracy_score
from generate_prompt import extract_author_dblp_ids
from utils import preprocess_text

# Load data
with open("DBLP-QuAD/DBLP-QuAD/test/questions.json", "r") as file:
    questions = json.load(file)

def pad_lists(list1, list2, pad_value=""):
    max_len = max(len(list1), len(list2))
    return list1 + [pad_value] * (max_len - len(list1)), list2 + [pad_value] * (max_len - len(list2))

# Store all accuracies
accuracies = []
results = []

# Loop over questions
for question in questions["questions"]:
    entities = [re.sub(r'[<>]', '', entity) for entity in question["entities"]]
    nl_query = preprocess_text(question["question"]["string"])
    predicted_entities = list(extract_author_dblp_ids(nl_query).values())

    # Pad lists to match lengths
    entities, predicted_entities = pad_lists(entities, predicted_entities)

    # Compute accuracy
    acc = accuracy_score(entities, predicted_entities)
    accuracies.append(acc)

    # Save result
    results.append({
        "question": nl_query,
        "ground_truth": entities,
        "predicted": predicted_entities,
        "accuracy": acc
    })

    print(f"✅ Accuracy for question: {acc:.2f}")

# Compute average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print(f"\n📊 Average Accuracy: {average_accuracy:.4f}")

# Save all results
with open("evaluation_results.json", "w") as f:
    json.dump(results, f, indent=2)


c:\Users\Administrator\kg-qa\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Robert Schober
✅ Accuracy for question: 1.00
S. Rodríguez
✅ Accuracy for question: 0.00
Mohammad Soleymani
✅ Accuracy for question: 0.00
D. Neamati
✅ Accuracy for question: 1.00
H. Kawasaki
✅ Accuracy for question: 1.00
Pablo G. Tahoces
✅ Accuracy for question: 1.00
Sameh S. Askar
✅ Accuracy for question: 1.00
Matti R.
✅ Accuracy for question: 0.00
David S. Oliveira
✅ Accuracy for question: 1.00
José Joaquín Bernal
✅ Accuracy for question: 1.00
Peter W. Glynn
✅ Accuracy for question: 1.00
Omer L.
✅ Accuracy for question: 0.00
Loukmen Regainia
✅ Accuracy for question: 1.00
Christian L. Müller
✅ Accuracy for question: 1.00
Dimitri K.
✅ Accuracy for question: 0.00
H. Hai Pham
✅ Accuracy for question: 0.00
Stefan Schreiber
✅ Accuracy for question: 1.00
Omar El-Gayar
✅ Accuracy for question: 1.00
Joaquin Vanschoren
✅ Accuracy for question: 1.00
✅ Accuracy for question: 0.00
Goele Pipeleers
✅ Accuracy for question: 1.00
Daniel S. Weld
✅ Accuracy for question: 1.00
Christian A. Larsson
✅ Accu

In [8]:
import requests

question = "Which papers did the authors Yin, Jianqin and Zeng, X. co-write?"
url = "https://ltdemos.informatik.uni-hamburg.de/dblplinkapi/api/entitylinker/t5-small/distmult"

headers = {
    "Content-Type": "application/json"
}

payload = {
    "question": question
}

response = requests.post(url, json=payload, headers=headers)

print("Status:", response.status_code)
print("Response:", response.text)



Status: 200
Response: {"entitylinkingresults":[{"label":"Jianqin Yin","result":[[-1,["<https://dblp.org/pid/21/2631>","Jianqin Yin"]],[-1,["<https://dblp.org/pid/219/4096>","Jianqin Feng"]],[-1,["<https://dblp.org/pid/219/9850>","Jianqin Fu"]],[-1,["<https://dblp.org/pid/151/9195>","Jianqin Zhao"]],[-1,["<https://dblp.org/pid/210/6707>","Jianqin Cao"]],[-1,["<https://dblp.org/pid/313/3825>","Jianqin Hang"]],[-1,["<https://dblp.org/pid/13/7485>","Jianqin Sun"]],[-1,["<https://dblp.org/pid/93/7636>","Jianqin Xie"]],[-1,["<https://dblp.org/pid/14/6236>","Jianqin Ding"]]],"type":["Creator","Person"]},{"label":"Xiao Zeng","result":[[-1,["<https://dblp.org/pid/83/2795>","Xiao Zeng"]],[-1,["<https://dblp.org/pid/46/466>","Xiao-Ming Zeng"]],[-1,["<https://dblp.org/pid/02/2865>","Xiao-Zeng You"]],[-1,["<https://dblp.org/pid/52/8125>","Xiao-hui Zeng"]],[-1,["<https://dblp.org/pid/22/6805>","Zeng-Xiao Guo"]],[-1,["<https://dblp.org/pid/286/5212>","Xiao S. Zeng"]],[-1,["<https://dblp.org/pid/311/0

In [ ]:
import json
import requests

with open("SciQA-dataset/SciQA-dataset/test/questions.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    
prefixes = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX orkgc: <http://orkg.org/orkg/class/>
PREFIX orkgp: <http://orkg.org/orkg/predicate/>
PREFIX orkgr: <http://orkg.org/orkg/resource/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
"""

question = data["questions"][0]

query_body = question["query"]["sparql"]

full_query = prefixes + "\n" + query_body

In [15]:
print(full_query)


PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX orkgc: <http://orkg.org/orkg/class/>
PREFIX orkgp: <http://orkg.org/orkg/predicate/>
PREFIX orkgr: <http://orkg.org/orkg/resource/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?paper ?paper_lbl
WHERE {
  ?dataset        a                orkgc:Dataset;
                  rdfs:label       ?dataset_lbl.
  FILTER (str(?dataset_lbl) = "Penn Treebank (Word Level)")
  ?benchmark      orkgp:HAS_DATASET       ?dataset.
  ?cont          orkgp:HAS_BENCHMARK      ?benchmark.
  ?paper         orkgp:P31                ?cont;
                 rdfs:label               ?paper_lbl.
}


In [18]:
import requests

query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX orkgc: <http://orkg.org/orkg/class/>
PREFIX orkgp: <http://orkg.org/orkg/predicate/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?properties_values ?property_description
WHERE {
  ?papers rdf:type orkgc:Paper .
  ?papers rdfs:label ?papers_labels .
  FILTER(REGEX(?papers_labels, "Elorza crater", "i"))
  ?papers orkgp:P31 ?contrib .
  ?contrib ?properties ?properties_values .
  ?properties rdfs:label ?properties_labels .
  FILTER(REGEX(?properties_labels, "FeO"))
  ?properties orkgp:description ?property_description .
}
"""

url = "http://localhost:7200/repositories/dblp_kg"
headers = {
    "Accept": "application/sparql-results+json"
}

response = requests.get(url, params={'query': full_query}, headers=headers)

if response.status_code == 200:
    data = response.json()
    for binding in data["results"]["bindings"]:
        print(binding)
else:
    print(f"Error {response.status_code}: {response.text}")



{'model': {'type': 'uri', 'value': 'http://orkg.org/orkg/resource/R117327'}, 'model_lbl': {'type': 'literal', 'value': 'GPT-3 175B (Few-Shot)'}}


In [2]:
import json
import requests

with open("SciQA-dataset/SciQA-dataset/test/questions.json", "r", encoding="utf-8") as f:
    data = json.load(f)

url = "https://orkg.org/sparql/"

# Common prefixes to prepend
prefixes = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX orkgc: <http://orkg.org/orkg/class/>
PREFIX orkgp: <http://orkg.org/orkg/predicate/>
PREFIX orkgr: <http://orkg.org/orkg/resource/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
"""

# Loop through each query and run it
#question = data["questions"][1]

#query_body = question["query"]["sparql"]
#query_body = "SELECT ?properties_values ?property_description\nWHERE {\n  ?papers rdf:type orkgc:Paper.\n  ?papers rdfs:label ?papers_labels.\n  FILTER(REGEX(?papers_labels, \"Elorza crater\", \"i\"))\n  ?papers orkgp:P31 ?contrib.\n  ?contrib ?properties ?properties_values.\n  ?properties rdfs:label ?properties_labels.\n  FILTER(REGEX(?properties_labels, \"FeO\"))\n  ?properties orkgp:description ?property_description.\n}"
query_body = """
SELECT ?properties_values ?property_description
WHERE {
  ?papers rdf:type orkgc:Paper ;
          rdfs:label ?papers_labels ;
          orkgp:P31 ?contrib.
          
  FILTER(REGEX(?papers_labels, "Elorza crater", "i"))

  ?contrib ?properties ?properties_values.
  ?properties rdfs:label ?properties_labels ;
              orkgp:description ?property_description.

  FILTER(REGEX(?properties_labels, "FeO"))
}
"""

full_query = prefixes + "\n" + query_body
query= """
PREFIX onto: <http://www.ontotext.com/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX orkgc: <http://orkg.org/orkg/class/>
PREFIX orkgp: <http://orkg.org/orkg/predicate/>
PREFIX orkgr: <http://orkg.org/orkg/resource/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?properties_values ?property_description
WHERE {
  ?papers rdf:type orkgc:Paper.
  ?papers rdfs:label ?papers_labels.
  FILTER(REGEX(?papers_labels, "Elorza crater", "i"))
  ?papers orkgp:P31 ?contrib.
  ?contrib ?properties ?properties_values.
  ?properties rdfs:label ?properties_labels.
  FILTER(REGEX(?properties_labels, "FeO"))
  ?properties orkgp:description ?property_description.
}
"""

    
response = requests.get(
        url,
        data={"query": query},
        headers={"Accept": "application/sparql-results+json"},
        verify=False  # set to True if your environment supports certs
    )

c:\Users\Administrator\kg-qa\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'orkg.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
import json
import requests

with open("SciQA-dataset/SciQA-dataset/test/questions.json", "r", encoding="utf-8") as f:
    data = json.load(f)

url = "https://orkg.org/sparql/"

# Common prefixes to prepend
prefixes = """
PREFIX onto: <http://www.ontotext.com/>
PREFIX orkgc: <http://orkg.org/orkg/class/>
PREFIX orkgp: <http://orkg.org/orkg/property/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
"""

# Loop through each query and run it
for question in data["questions"]:
    query_body = question["query"]["sparql"]
    full_query = prefixes + "\n" + query_body

    try:
        response = requests.get(
            url,
            params={"query": full_query},
            headers={"Accept": "application/sparql-results+json"},
            verify=False  # set to True if your environment supports certs
        )
        response.raise_for_status()

        results = response.json()

        print(f"\n=== Results for Question ID: {question['id']} ===")
        for binding in results["results"]["bindings"]:
            row = {k: v["value"] for k, v in binding.items()}
            print(row)

    except Exception as e:
        print(f"Error for question {question['id']}: {e}")
